In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    streaming=False, # maxA_token_limit 초과하면 이전 메지지가 요약된다. 
                     # 요약 메시지를 출력하지 않기 위해서 False로 설정
    callbacks=[StreamingStdOutCallbackHandler()],
    model="gpt-3.5-turbo",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True,
)

examples = [
    {
        "movie": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "인터스텔라",
        "answer": "🚀🧑‍🚀🌌",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Tell me 3 icons representing {movie}"),
    ("ai", "{answer}")
])
intermediate_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a film expert, you should give short answers"),
    intermediate_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "Tell me 3 icons representing {movie}"),
])


def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
  result = chain.invoke({"movie": movie})
  memory.save_context({"input": movie}, {"output": result.content})
  print(result.content)
  
  

In [35]:
invoke_chain("기생충")

🏠🐜💰


In [36]:
invoke_chain("블레이드러너")

🌃👾🤖


In [37]:
memory.load_memory_variables({})


{'history': [HumanMessage(content='기생충', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🏠🐜💰', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='블레이드러너', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🌃👾🤖', additional_kwargs={}, response_metadata={})]}

In [38]:
invoke_chain("명량")

🚢💥🌊


In [39]:
invoke_chain("극한직업")

👮‍♂️🍗🔥


In [40]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions "Parasite." The AI responds with emojis indicating a house, ants, and money. The human then brings up "Blade Runner."', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🌃👾🤖', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='명량', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🚢💥🌊', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='극한직업', additional_kwargs={}, response_metadata={}),
  AIMessage(content='👮\u200d♂️🍗🔥', additional_kwargs={}, response_metadata={})]}